In [2]:
!pip install transformers


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.5.1
    Uninstalling dill-0.3.5.1:
      Successfully uninstalled dill-0.3.5.1

[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from datasets import load_dataset, load_metric
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          Trainer, TrainingArguments)

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
dataset = load_dataset('glue', 'mrpc')
metric = load_metric('glue', 'mrpc')

def encode(examples):
    outputs = tokenizer(
        examples['sentence1'], examples['sentence2'], truncation=True)
    return outputs

encoded_dataset = dataset.map(encode, batched=True)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        'distilbert-base-uncased', return_dict=True)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Evaluate during training and a bit more often
# than the default to be able to prune bad trials early.
# Disabling tqdm is a matter of preference.
training_args = TrainingArguments(
    "test", evaluation_strategy="steps", eval_steps=500, disable_tqdm=True)
trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    model_init=model_init,
    compute_metrics=compute_metrics,
)

# Default objective is the sum of all metrics
# when metrics are provided, so we have to maximize it.
trainer.hyperparameter_search(
    direction="maximize", 
    backend="ray", 
    n_trials=10 # number of trials
)

/home/ray/anaconda3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset glue downloaded and prepared to /home/jovyan/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 947.08it/s]
/tmp/ipykernel_44/2149597044.py:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'mrpc')
100%|██████████| 2/2 [00:00<00:00, 32.07ba/s]
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/jovyan/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2023-02-20 23:13:11,327	WARNING utils.py:594 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.
2023-02-20 23:13:11,578	WARNING services.py:1882 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67104768 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.


...

(_objective pid=956) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias']
(_objective pid=956) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=956) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=956) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased

(_objective pid=869) {'train_runtime': 22.4566, 'train_samples_per_second': 490.011, 'train_steps_per_second': 20.439, 'train_loss': 0.5574849114179091, 'epoch': 3.0}
Result for _objective_34426_00007:
  date: 2023-02-20_23-14-51
  done: true
  epoch: 3.0
  eval_accuracy: 0.7132352941176471
  eval_f1: 0.821917808219178
  eval_loss: 0.5029550194740295
  eval_runtime: 0.3661
  eval_samples_per_second: 1114.549
  eval_steps_per_second: 139.319
  experiment_id: f2637d22e92447e8b0600cf2edc7fb12
  hostname: 3b3c85b4f075
  iterations_since_restore: 1
  node_ip: 192.168.48.4
  objective: 1.535153102336825
  pid: 869
  time_since_restore: 26.227601528167725
  time_this_iter_s: 26.227601528167725
  time_total_s: 26.227601528167725
  timestamp: 1676963691
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '34426_00007'
  warmup_time: 0.0021789073944091797
  
(_objective pid=869) {'eval_loss': 0.5029550194740295, 'eval_accuracy': 0.7132352941176471, 'eval_f1': 0.821917808219178, 'eva

2023-02-20 23:15:05,645	INFO tune.py:758 -- Total run time: 112.52 seconds (112.29 seconds for the tuning loop).


Result for _objective_34426_00009:
  date: 2023-02-20_23-15-05
  done: true
  epoch: 2.0
  eval_accuracy: 0.7009803921568627
  eval_f1: 0.8140243902439025
  eval_loss: 0.5210139751434326
  eval_runtime: 0.3137
  eval_samples_per_second: 1300.652
  eval_steps_per_second: 162.582
  experiment_id: f26a15f2acc740f6a26eea7ac9c8742d
  hostname: 3b3c85b4f075
  iterations_since_restore: 1
  node_ip: 192.168.48.4
  objective: 1.5150047824007653
  pid: 956
  time_since_restore: 19.2745418548584
  time_this_iter_s: 19.2745418548584
  time_total_s: 19.2745418548584
  timestamp: 1676963705
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '34426_00009'
  warmup_time: 0.0025255680084228516
  
== Status ==
Current time: 2023-02-20 23:15:05 (running for 00:01:52.30)
Memory usage on this node: 11.0/251.6 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/3 GPUs, 0.0/46.31 GiB heap, 0.0/9.31 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/jovyan/ray_resu

BestRun(run_id='34426_00005', objective=1.7267488076311608, hyperparameters={'learning_rate': 1.1207606211860595e-05, 'num_train_epochs': 4, 'seed': 1.8994345766152145, 'per_device_train_batch_size': 16})